In [45]:
import pickle
import json
import matplotlib.pyplot as plt
import glob
import numpy as np
import datetime
import pandas as pd
from scipy import stats
import pytz
from geo_utils import calculate_dist
from collections import Counter
plt.style.use('seaborn-poster')
%matplotlib inline
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [14]:
data_path = '../data/myshake_data/20160212_20170101/*_ground_truth.pkl'
simulation_path = '../data/simulation_data/'

In [4]:
def get_p_and_s(max_dist_km, evdp):
    
    ep_dist_km = np.arange(max_dist_km)
    
    hyp = np.sqrt(ep_dist_km**2 + evdp**2)
    
    return ep_dist_km, hyp/6.10, hyp/3.55

## Read in the EPIC triggers

In [6]:
df_epic = pd.read_hdf('../data/e2trigs_20160201_20170101.h5')

In [44]:
for f_hb in glob.glob(data_path):
    
    evid = os.path.basename(f_hb).split('_')[0]
    
    if evid != 'ci37374687':
        continue
    
    simulation_file = glob.glob(os.path.join(simulation_path, evid + '*'))[0]
    simulation_dict = pickle.load(open(simulation_file, 'rb'))
    
    i = 0    
    #if os.path.isfile('../figures/simulation_figures/' + evid + '.png'):
    #    continue
    

    # read in all the phones
    phones = simulation_dict[i]['phones']
    steady_phones = simulation_dict[i]['phones_steady']
    df_trig = simulation_dict[i]['df_trig']
    earthquake = pd.read_pickle(f_hb)
    
    mag = earthquake['mag']
    evla = earthquake['latitude']
    evlo = earthquake['longitude']
    evdp = earthquake['depth']
    evtime = earthquake['time']
    place = earthquake['place']
    
    trig_times = []
    dists = []

    # origin time in timestamp
    orig_time = (datetime.datetime.strptime(evtime.replace('Z',''), '%Y-%m-%dT%H:%M:%S.%f') - \
                         datetime.datetime(1970, 1, 1)).total_seconds()

    # get the trigger distance km
    triggers_dist = df_trig['dist']

    # get the total phones distance km
    total_phones_dist = []
    for stla, stlo in phones:
        total_phones_dist.append(calculate_dist(evla, evlo, stla, stlo))
    total_phones_dist = np.array(total_phones_dist)
    
    # get the steady phones distance km
    steady_phones_dist = []
    for stla, stlo in steady_phones:
        steady_phones_dist.append(calculate_dist(evla, evlo, stla, stlo))
    steady_phones_dist = np.array(steady_phones_dist)
    
    # within 100 km
    total_phones_dist_select = total_phones_dist[total_phones_dist < 100]
    steady_phones_dist_select = steady_phones_dist[steady_phones_dist < 100]
    triggers_dist_select = triggers_dist[triggers_dist<100]
    
    # triggers within 100 km
    df_trig = df_trig[df_trig['dist']<100]
    
    ep_dist_km, tp_sec, ts_sec = get_p_and_s(500, evdp)
    
    # get the EPIC triggers
    t_start = evtime[:-5]
    t_end = datetime.datetime.strptime(evtime.replace('Z',''), '%Y-%m-%dT%H:%M:%S.%f') \
            + datetime.timedelta(seconds=60)
    t_end = t_end.strftime('%Y-%m-%dT%H:%M:%S')
    
    t_30s_before = datetime.datetime.strptime(evtime.replace('Z',''), '%Y-%m-%dT%H:%M:%S.%f') \
            - datetime.timedelta(seconds=30)
    t_30s_before = t_30s_before.strftime('%Y-%m-%dT%H:%M:%S')
    
    df_epic_select = df_epic[t_start:t_end]
    
    epic_trig_time_s = []
    epic_trig_dist_km = []
    for ix, row in df_epic_select.iterrows():
        epic_trig_tt = row['time'] - orig_time
        epic_trig_dist = calculate_dist(evla, evlo, row['lat'], row['lon'])
        epic_trig_time_s.append(epic_trig_tt)
        epic_trig_dist_km.append(epic_trig_dist)
        
    df_epic_select = df_epic_select.assign(trigT_s=epic_trig_time_s)
    df_epic_select = df_epic_select.assign(dist_km=epic_trig_dist_km)
    
    df_epic_select_100 = df_epic_select[df_epic_select['dist_km'] <100][t_30s_before:t_end]
    
    df_trig_300 = df_trig[t_start:t_end]
    
    
    # distance bins
    try:
        total_sum, edge, _ = stats.binned_statistic(total_phones_dist_select, [1]*len(total_phones_dist_select), 
                       statistic='sum', bins=np.arange(0, 105, 5))
    except:
        total_sum = np.zeros(len(edge[1:]))

    try:
        steady_sum, edge, _ = stats.binned_statistic(steady_phones_dist_select, [1]*len(steady_phones_dist_select), 
                       statistic='sum', bins=np.arange(0, 105, 5))
    except:
        steady_sum = np.zeros(len(edge[1:]))
    
    try:
        trig_sum, edge, _ = stats.binned_statistic(triggers_dist_select, [1]*len(triggers_dist_select), 
                       statistic='sum', bins=np.arange(0, 105, 5))
    except:
        trig_sum = np.zeros(len(edge[1:]))
        
    try:
        epic_sum, edge, _ = stats.binned_statistic(df_epic_select['dist_km'], [1]*len(df_epic_select['dist_km']), 
                       statistic='sum', bins=np.arange(0, 105, 5))
    except:
        epic_sum = np.zeros(len(edge[1:]))
    
    # time bins
    df_trig_300 = df_trig[t_30s_before:t_end]
    
    try:
        trig_t_sum, edge_t, _ = stats.binned_statistic(df_trig_300['delta_t'].values, [1]*len((df_trig_300['delta_t'])), 
                       statistic='sum', bins=np.arange(-30, 61, 1))
    except:
        trig_t_sum = np.zeros(len(edge_t[1:]))
        
    try:
        epic_t_sum, edge_t, _ = stats.binned_statistic(df_epic_select_100['trigT_s'].values, [1]*len(df_epic_select_100['trigT_s']), 
                       statistic='sum', bins=np.arange(-30, 61, 1))
    except:
        epic_t_sum = np.zeros(len(edge_t[1:]))


    
    fig = plt.figure(figsize=(16, 10))
    
    fig.suptitle(f'M{mag} event at depth {evdp} km, on {evtime[:-5]}, at {place}', 
                 fontsize=20)

    gs = gridspec.GridSpec(3, 4)

    ax1 = plt.subplot(gs[0:2,0:3])

    ax1.plot(ep_dist_km, tp_sec, 'g', label='P-wave')
    ax1.plot(ep_dist_km, ts_sec, 'r', label='S-wave')
    
    ax1.scatter(df_trig_300['dist'], df_trig_300['delta_t'],s=30, c='k', 
                label='Simulated phone triggers', marker='.', zorder=9)
    
    ax1.set_xticklabels([])


    ax1.scatter(df_epic_select['dist_km'], df_epic_select['trigT_s'],s=60, c='g', 
                label='EPIC triggers', marker='s', zorder=10)

    ax1.set_ylabel('Time since origin of the earthquake (sec)')
    ax1.set_ylim(-30, 70)
    ax1.set_xlim(0, 100)
    plt.legend(loc=4)

    #ax2 = fig.add_subplot(1, 2, 2, projection=ccrs.PlateCarree())
    ax2 = plt.subplot(gs[2,3:], projection=ccrs.PlateCarree())
    ax2.set_extent([-125, -114, 32, 42], crs=ccrs.PlateCarree())

    ax2.add_feature(cfeature.LAND)
    ax2.add_feature(cfeature.OCEAN)
    ax2.add_feature(cfeature.COASTLINE)
    ax2.add_feature(cfeature.BORDERS, linestyle=':')
    ax2.add_feature(cfeature.LAKES, alpha=0.5)
    ax2.add_feature(cfeature.RIVERS)
    ax2.add_feature(cfeature.STATES)
    ax2.scatter(evlo, evla, marker='*', s=80, c='r')
    
    ax3 = plt.subplot(gs[2,0:3])
    ax3.bar(edge[1:], total_sum, width = 5, color='b', label='PHONE')
    #ax3.bar(edge[1:], steady_sum, width = 4, color='c', label='STEADY' )
    ax3.bar(edge[1:], trig_sum, width = 4, color='k', label='TRIGGER')
    ax3.bar(edge[1:], epic_sum, width = 3, color='g', label='EPIC')
    ax3.set_xlim(0, 100)
    plt.legend()
    plt.xlabel('Epicentral distance (km)')
    
    ax4 = plt.subplot(gs[:2,3:])
    
    ax4.barh(edge_t[1:], trig_t_sum, color='k', height=0.8, label='Phone' )
    ax4.barh(edge_t[1:], epic_t_sum, color='g', height=1, label='EPIC')
    
    ax4.set_ylim(-30, 70)
    ax4.set_yticklabels([])
    plt.legend(loc=4)
    
    
    plt.tight_layout()
    plt.savefig('../figures/simulation_figures/' + evid + '.png', transparent = False, 
                bbox_inches = 'tight', pad_inches = 0.1)
    plt.close()
    #plt.show()
    
    #break

/Users/qingkaikong/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:189: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
